In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [2]:
import skimage, os
from skimage.morphology import ball, disk, dilation,binary_dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing, binary_opening 
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border, mark_boundaries
from skimage import data
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
from glob import glob
from skimage.io import imread


In [3]:
covidx_dir = '/media/ubuntu/MyHDataStor3/datasets/COV19D/'
covidx_img_dir= covidx_dir 
covidx_mask_dir= covidx_dir + 'mask/' 
data_list_dir = '/media/ubuntu/MyHDataStor2/products/COVID-19/ICCV-MAI/3D-CNN-BERT/BERT/datasets/settings/covid/'

print(covidx_dir)
print(covidx_img_dir)
print(covidx_mask_dir)
print(data_list_dir)

/media/ubuntu/MyHDataStor3/datasets/COV19D/
/media/ubuntu/MyHDataStor3/datasets/COV19D/
/media/ubuntu/MyHDataStor3/datasets/COV19D/mask/
/media/ubuntu/MyHDataStor2/products/COVID-19/ICCV-MAI/3D-CNN-BERT/BERT/datasets/settings/covid/


In [4]:
def seedfill(img):

  rows,cols = img.shape 

  img_floodfill = img.copy()
  mask = np.zeros((rows+2,cols+2), np.uint8)

  '''
  # Floodfill from point (0, 0)
  found  = False 
  for x in range(cols): 
    for y in range(rows):
      if img_floodfill[y,x] == 0:
        found = True 
        break  
    if found:
      break 

  #cv2.floodFill(img_floodfill, mask, (x,y), 255);
  ''' 
  

  cv2.floodFill(img_floodfill, mask, (0,0), 255);
  cv2.floodFill(img_floodfill, mask, (0,rows-1), 255);
  cv2.floodFill(img_floodfill, mask, (cols-1,0), 255);
  cv2.floodFill(img_floodfill, mask, (cols-1,rows-1), 255);

  return img_floodfill 


In [5]:
def body_contour(binary_image):
    """Helper function to get body contour"""
    contours, hier = cv2.findContours(binary_image, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in contours]
    body_idx = np.argmax(areas)
    return contours[body_idx]


def auto_body_crop(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(3, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    #print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)         
    
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
    
    
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    '''
    #find the lung rectangle        
    binary2 = ((1-bin_image)*255).astype(np.uint8) 
    kernel = np.ones((5,5), np.uint8) 
    binary2 = cv2.dilate(binary2, kernel, iterations=2) 
    binary2 = cv2.erode(binary2, kernel, iterations=2) 
    binary2 = seedfill(binary2) 
    binary2 = cv2.bitwise_not(binary2)    
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(binary2, cmap=plt.cm.bone) 
    '''    
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
    return bin_image, (xmin, ymin, xmax, ymax)


def auto_segment(image, plot=False):
    """Roughly crop an image to the body region"""
    # Create initial binary image
    filt_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    if plot == True:
        f, plots = plt.subplots(9, 1, figsize=(5, 40))        
        plots[0].axis('off')
        plots[0].imshow(filt_image, cmap=plt.cm.bone)     
    
    
    thresh = cv2.threshold(filt_image[filt_image > 0], 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[0]
    print(thresh) 
    
    bin_image = np.uint8(filt_image > thresh)  
    #bin_image2 = np.uint8(filt_image > 240)             
    
    if plot == True:
        plots[1].axis('off')
        plots[1].imshow(bin_image, cmap=plt.cm.bone)     
        
    erode_kernel = np.ones((7, 7), dtype=np.uint8)
    bin_image = cv2.erode(bin_image, erode_kernel)
    #print(np.unique(bin_image))
    if plot == True:
        plots[2].axis('off')
        plots[2].imshow(bin_image, cmap=plt.cm.bone)     
        
 
        
    # Find body contour
    body_cont = body_contour(bin_image).squeeze()
    
    #print(body_cont)

    # Get bbox
    xmin = body_cont[:, 0].min()
    xmax = body_cont[:, 0].max() + 1
    ymin = body_cont[:, 1].min()
    ymax = body_cont[:, 1].max() + 1
       
        
    bin_image_inv = (1 - bin_image)*255 
    bin_image_inv = seedfill(bin_image_inv)/255 
    cleared = clear_border(bin_image_inv)
    if plot == True:
        plots[3].axis('off')
        plots[3].imshow(cleared, cmap=plt.cm.bone)     
    
    label_image = label(cleared)
    if plot == True:
        plots[4].axis('off')
        plots[4].imshow(label_image, cmap=plt.cm.bone) 
        
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       label_image[coordinates[0], coordinates[1]] = 0

    binary = label_image > 0
    if plot == True:
        plots[5].axis('off')
        plots[5].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 5: Erosion operation with a disk of radius 2. This operation is 
    seperate the lung nodules attached to the blood vessels.
    '''
    selem = disk(7)
    binary = binary_erosion(binary, selem)
    if plot == True:
        plots[6].axis('off')
        plots[6].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 6: Closure operation with a disk of radius 10. This operation is 
    to keep nodules attached to the lung wall.
    '''
    selem = disk(10)
    binary = binary_closing(binary, selem)
    if plot == True:
        plots[7].axis('off')
        plots[7].imshow(binary, cmap=plt.cm.bone) 
    '''
    Step 7: Fill in the small holes inside the binary mask of lungs.
    '''

    binary1 = binary.copy() 

    edges = roberts(binary)
    binary = ndi.binary_fill_holes(edges)
    if plot == True:
        plots[8].axis('off')
        plots[8].imshow(binary, cmap=plt.cm.bone) 
    
    
    return binary, binary1, (xmin, ymin, xmax, ymax)


In [6]:
def refine_mask(m_full):
    
    mask2 = cv2.imread(m_full, cv2.IMREAD_UNCHANGED)
    mask2 = (mask2/255).astype(np.uint8) 
    
    selem = disk(10)
    mask2_2  = binary_closing(mask2, selem)                
    label_image = label(mask2_2)
    
    '''
    Step 4: Keep the labels with 2 largest areas.
    '''
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    if len(areas) > 2:
        for region in regionprops(label_image):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                    label_image[coordinates[0], coordinates[1]] = 0
    mask2_3 = label_image > 0               
    
    edges = roberts(mask2_3) 
    mask2_4 = ndi.binary_fill_holes(edges)
                
    return mask2,mask2_4

In [7]:
#subsets = ['val','train']  #for test dataset, UNet segmentation and refinement are combined. 
subsets = ['train']

from tqdm import tqdm

debug = False
split = 1 

for subset in subsets:     

    list_file = data_list_dir + '{}_rgb_split{}.txt'.format(subset,split) 
    lines = open(list_file,"r").readlines() 

    pbar = tqdm(total = len(lines))
    
    for ind in range(len(lines)-1,-1,-1):
        
        l = lines[ind]
       
        print(l)
        
        pbar.update()
        
        l_splits = l.split()  
                
        l_path = l_splits[0]        
        c = l_splits[1]
        first_slice = int(l_splits[2])
        last_slice = int(l_splits[3])

        count = 0 
        for f in range(first_slice,last_slice+1):
            
            count +=1 
            
            #if count>10:
            #    break 
            
            fn = "{}/{}.jpg".format(l_path,f)
            f_full = covidx_img_dir + fn
            m_full = covidx_img_dir + 'mask/' + fn
            m_full_new = m_full+'_refined.jpg'            
            #print(f_full,m_full,m_full_new)
            
            if not os.path.exists(m_full_new) :
                mask0,mask = refine_mask(m_full)                
                cv2.imwrite(m_full_new, (mask*255).astype(np.uint8) )

                if False: 
                    fig, (ax1,ax2) = plt.subplots(1,2)
                    ax1.imshow(mask0*255,cmap='gray')
                    ax2.imshow(mask*255,cmap='gray')                
                


  0%|          | 0/1992 [00:00<?, ?it/s]

train/covid/subset3/ct_scan_498 covid 22 69 0 73 512 419



  0%|          | 2/1992 [00:04<1:07:24,  2.03s/it]

train/covid/subset4/ct_scan832 covid 6 24 0 129 512 422



  0%|          | 3/1992 [00:05<1:03:18,  1.91s/it]

...

 78%|███████▊  | 1556/1992 [1:19:35<07:02,  1.03it/s]

train/non-covid/subset3/ct_scan_13 non-covid 49 113 0 44 512 443

train/non-covid/subset5/ct_scan1030 non-covid 11 24 0 118 512 442

train/non-covid/subset5/ct_scan945 non-covid 230 305 40 63 477 432

train/non-covid/subset4/ct_scan_556 non-covid 28 55 0 53 512 416

train/non-covid/subset5/ct_scan963 non-covid 129 263 0 88 512 512

train/non-covid/subset5/ct_scan1092 non-covid 16 69 0 69 508 440

train/non-covid/subset2/ct_scan_335 non-covid 94 297 14 107 466 442

train/non-covid/subset4/ct_scan_461 non-covid 10 17 0 138 512 402

train/non-covid/subset1/ct_scan_207 non-covid 91 185 5 73 512 455

train/non-covid/subset1/ct_scan_292 non-covid 92 196 0 27 512 482

train/non-covid/subset4/ct_scan_593 non-covid 0 81 0 65 512 434

train/non-covid/subset4/ct_scan_864 non-covid 25 50 0 7 512 408

train/non-covid/subset4/ct_scan_539 non-covid 18 27 6 137 512 380

train/non-covid/subset3/ct_scan_175 non-covid 22 41 0 116 512 453

train/non-covid/subset4/ct_scan_731 non-covid 61 210 0 97 512 489


 97%|█████████▋| 1930/1992 [1:19:35<00:29,  2.10it/s]

train/non-covid/subset4/ct_scan_737 non-covid 40 83 11 70 508 422

train/non-covid/subset5/ct_scan953 non-covid 73 220 0 41 512 442

train/non-covid/subset4/ct_scan_830 non-covid 20 62 0 51 512 432

train/non-covid/subset3/ct_scan_124 non-covid 15 35 0 75 512 490

train/non-covid/subset2/ct_scan_371 non-covid 20 38 0 25 512 464

train/non-covid/subset3/ct_scan_112 non-covid 13 24 0 69 512 442

train/non-covid/subset1/ct_scan_213 non-covid 14 39 0 114 512 483

train/non-covid/subset2/ct_scan_336 non-covid 10 20 0 116 512 505

train/non-covid/subset5/ct_scan988 non-covid 49 81 0 78 512 434

train/non-covid/subset4/ct_scan_640 non-covid 17 25 13 133 512 406

train/non-covid/subset4/ct_scan_734 non-covid 74 209 0 110 512 478

train/non-covid/subset3/ct_scan_136 non-covid 95 144 0 90 512 467

train/non-covid/subset5/ct_scan960 non-covid 166 269 0 83 512 479

train/non-covid/subset3/ct_scan_143 non-covid 104 173 0 8 512 417

train/non-covid/subset4/ct_scan_487 non-covid 33 70 16 90 488 442



100%|██████████| 1992/1992 [1:19:50<00:00,  2.10it/s]